In [1]:
"""
    Image classification using convolutional neural network
    dataset: CIFAR-10
    Framework: Tensorflow
"""

import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import cv2
from IPython.display import clear_output

In [2]:
label_list = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# creating one-hot matrix of the output labels
def one_hot(base):
    # one_hot matrix
    hot_matrix = np.zeros([np.size(base[:, 0]), 10], dtype = float)
    for i in range(np.size(base[:, 0])):
        word = base[i, 1]
        hot_matrix[i, label_list.index(word)] = 1
    return hot_matrix

In [3]:
# creating minibatches
def mini_batch(X, Y, size):
    idx = np.random.randint(np.size(Y[:, 0]), size = (size,1))
    x_bat = X[idx]
    x_bat = x_bat.reshape(size, 32, 32, 3)
    y_bat = Y[idx]
    y_bat = y_bat.reshape(size, 10)
    
    return x_bat, y_bat

In [4]:
# importing 10000 images of size 32x32x3
i = 0
img_base = []
for img in glob.glob("C:\\Users\\krmmv\\Documents\\jupyter\\train\\*.png"):
    img_base.append(cv2.imread(img))
    
    i+=1
    if i%1000 == 0:
        clear_output()
        print("iteration: ", i)
    
x_train = np.array(img_base[0:40000]).astype(np.float32)

iteration:  50000


In [5]:
# importing labels
labels = pd.DataFrame(pd.read_csv("trainLabels.csv"))
y_train = np.array(labels[0:40000])
y_train = one_hot(y_train)
print('training label shape: ',y_train.shape)
print('training images size: ', x_train.shape)

training label shape:  (40000, 10)
training images size:  (40000, 32, 32, 3)


In [6]:
# to reset the graph
tf.reset_default_graph()

# creating placeholders
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.float32, [None, 10])

# creating weights
w1 = tf.get_variable('w1', [3,3,3,10], initializer=tf.contrib.layers.xavier_initializer())
w2 = tf.get_variable('w2', [3,3,10,15], initializer=tf.contrib.layers.xavier_initializer())

In [7]:
# hyperparameters
learning_rate = 0.05

In [8]:
# forward propagation

# convolution layer 1
c1 = tf.nn.conv2d(x_train, w1, strides = [1,1,1,1], padding = 'SAME')
# activation function for c1: relu
r1 = tf.nn.relu(c1)
# maxpooling
p1 = tf.nn.max_pool(r1, ksize = [1,2,2,1], strides = [1,1,1,1], padding = 'SAME')
print('p1 shape: ',p1.shape)
# convolution layer 2
c2 = tf.nn.conv2d(p1, w2, strides = [1,1,1,1], padding='SAME')
# activation function for c2: relu
r2 = tf.nn.relu(c2)
# maxpooling
p2 = tf.nn.max_pool(r2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
print('p2 shape: ',p2.shape)
# fully connected layer
l1 = tf.contrib.layers.flatten(p2)
# fully connected layer
final = tf.contrib.layers.fully_connected(l1, 10, activation_fn = None)
print('output layer shape: ',final.shape)

p1 shape:  (40000, 32, 32, 10)
p2 shape:  (40000, 16, 16, 15)
output layer shape:  (40000, 10)


In [9]:
# training and optimization

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = final, labels = y_train))
# using adam optimizer 
optimize = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [ ]:
# creating tensorflow session
se = tf.Session()

# initializing variables
se.run(tf.global_variables_initializer())

# training the graph
for i in range(1000):
    x_batch, y_batch = mini_batch(x_train, y_train, 110)
    se.run(optimize, {x: x_batch, y: y_batch})
cost = se.run(cross_entropy, {x: x_train, y: y_train})
print(cost)

In [ ]:
# test set preparation
x_test = np.array(img_base[40000:]).astype(np.float32)
y_test = one_hot(np.array(labels[40000:]))

print('test input set size: ', x_test.shape)
print('test set labels size: ', y_test.shape)

# test set prediction
se.run(final, {x: x_test, y: y_test})